# 워드 임베딩 실습

- 딥러닝을 사용한 워드 임베딩
- N-Gram

## 1. 딥러닝을 사용한 워드 임베딩

In [3]:
#-*- coding: utf-8 -*-

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [4]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # the vocabulary size = 2 , dimensionality of the embeddings = 5 
lookup_tensor = torch.LongTensor([word_to_ix["hello"]])
hello_embed = embeds(autograd.Variable(lookup_tensor))
print(hello_embed)

Variable containing:
-2.9718  1.7070 -0.4305 -2.2820  0.5237
[torch.FloatTensor of size 1x5]



## 2. N-Gram

### 데이터(Shakespeare Sonnet 2)

In [5]:
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [6]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

In [7]:
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]

print(trigrams[:3])

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


### 모델

In [8]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out)
        return log_probs

### 학습 준비

In [9]:
vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [10]:
print(word_to_ix)

{'Shall': 0, 'warm': 1, 'all': 2, 'held:': 3, 'trenches': 4, 'much': 5, 'it': 6, 'And': 7, 'own': 8, 'old,': 9, 'thine!': 10, 'make': 11, 'treasure': 12, 'Then': 13, 'Where': 14, 'couldst': 15, 'within': 16, 'in': 17, 'were': 18, 'cold.': 19, 'praise.': 20, 'proud': 21, 'blood': 22, 'use,': 23, 'so': 24, "'This": 25, 'besiege': 26, 'thriftless': 27, 'shall': 28, 'lies,': 29, 'winters': 30, 'Proving': 31, "totter'd": 32, 'where': 33, 'say,': 34, 'my': 35, 'an': 36, 'on': 37, 'child': 38, 'worth': 39, "beauty's": 40, 'How': 41, 'praise': 42, 'his': 43, 'art': 44, "feel'st": 45, 'mine': 46, 'Will': 47, 'succession': 48, 'field,': 49, 'all-eating': 50, 'more': 51, 'the': 52, 'thy': 53, 'thine': 54, 'shame,': 55, 'be': 56, 'sum': 57, 'eyes,': 58, 'When': 59, 'Thy': 60, 'deep': 61, 'lusty': 62, 'by': 63, 'and': 64, 'fair': 65, 'answer': 66, 'sunken': 67, 'when': 68, 'Were': 69, 'forty': 70, 'count,': 71, 'brow,': 72, 'of': 73, 'made': 74, "deserv'd": 75, 'thou': 76, 'If': 77, 'This': 78, 'ol

### 학습
- 실습을 위해 iter를 조정합니다

In [14]:
for epoch in range(20):
    total_loss = torch.Tensor([0])
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in variables)
        context_idxs = [word_to_ix[w] for w in context]
        context_var = autograd.Variable(torch.LongTensor(context_idxs))

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_var)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a variable)
        loss = loss_function(log_probs, autograd.Variable(
            torch.LongTensor([word_to_ix[target]])))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        total_loss += loss.data
    losses.append(total_loss)

print(losses)  # The loss decreased every iteration over the training data!

[
 519.3522
[torch.FloatTensor of size 1]
, 
 517.0147
[torch.FloatTensor of size 1]
, 
 514.6943
[torch.FloatTensor of size 1]
, 
 512.3885
[torch.FloatTensor of size 1]
, 
 510.0958
[torch.FloatTensor of size 1]
, 
 507.8158
[torch.FloatTensor of size 1]
, 
 505.5493
[torch.FloatTensor of size 1]
, 
 503.2964
[torch.FloatTensor of size 1]
, 
 501.0559
[torch.FloatTensor of size 1]
, 
 498.8267
[torch.FloatTensor of size 1]
, 
 496.6071
[torch.FloatTensor of size 1]
, 
 494.3968
[torch.FloatTensor of size 1]
, 
 492.1956
[torch.FloatTensor of size 1]
, 
 490.0011
[torch.FloatTensor of size 1]
, 
 487.8141
[torch.FloatTensor of size 1]
, 
 485.6337
[torch.FloatTensor of size 1]
, 
 483.4597
[torch.FloatTensor of size 1]
, 
 481.2915
[torch.FloatTensor of size 1]
, 
 479.1304
[torch.FloatTensor of size 1]
, 
 476.9745
[torch.FloatTensor of size 1]
, 
 474.8228
[torch.FloatTensor of size 1]
, 
 472.6739
[torch.FloatTensor of size 1]
, 
 470.5281
[torch.FloatTensor of size 1]
, 
 468.3846

### 결과물

In [12]:
for param in model.parameters():
    print(param)

Parameter containing:
 0.8419  0.5512  0.3871  0.9134 -0.8409  1.2282 -1.8657  1.4142 -1.8785 -0.4687
-0.7577  0.4223 -0.4822 -1.1195  0.3057  1.0374  0.5202 -0.5010  1.2187  0.2106
-1.0640 -1.9448 -0.9589  0.5472 -0.9890 -0.3833  1.5045  1.8272  0.5541  1.6437
 0.4978 -1.5070  1.7673 -0.3574 -0.1696  0.4062 -0.4276 -1.1283  1.4272 -1.4026
 1.4831 -1.1568  1.6193  0.9586  0.7752  0.1948  0.1682  0.3063  1.0745 -1.0313
 1.0930  0.7783 -1.3134  0.7099  0.9952 -0.2707 -0.6485 -0.1365 -0.2953 -0.7731
-0.2218  0.5075 -0.6797 -1.6115  0.5227 -0.8885  0.2617  0.0303  0.0020 -1.3986
 1.4673 -0.1030 -0.0089 -0.8413 -0.2057  1.0665  0.1739 -0.6861  0.3119  0.2359
-1.0648  0.3628  0.3775 -0.2448 -0.5841  2.0831 -0.1183  0.4896  0.8347  0.8889
 0.4152  0.0515 -0.9640 -2.0121  0.5246  2.1330 -0.0822  0.8390 -1.3226  0.0709
 1.2214  0.4264 -1.2325 -0.6191  1.5139  1.9958 -0.6577 -0.4147 -0.2250 -0.6903
 0.9886  0.7419 -2.0984  1.2585 -0.3983 -1.0958 -1.0704  0.6409  1.6194  0.5258
-0.2960 -0.0689 -0